# Librerías

In [1]:
import json
import pandas as pd
import gcsfs
import lzma
import os
import csv

# Creación de la Base de Datos

Para esta versión del tratamiento de los datos se ha conseguido unificar los procesos que permiten la extracción de la información para un determinado tipo de influencer y de sector de forma que ahora **se ha generalizado el proceso** con la finalidad de recabar la totalidad del conjunto de datos.

### Selección de campos

El datalake que se desea estructurar va estar compuesto por **dos características importantes: sector y tipo de influencer**. 

Por un lado existe el sector de la belleza y coméstica frente al del mundo del fitness y la nutrición. Independientemente esto, la recopilación de la información puede seguir generalizada en los siguientes campos: 

In [2]:
def create_field_dict(node, owner):
    fields = {}
    fields['_id'] = node['id']
    fields['username'] = owner['username']
    fields['typename'] = node['__typename']
    fields['is_video'] = node['is_video']
    fields['followers']= owner['edge_followed_by']['count']
    fields['following'] = owner['edge_follow']['count']
    fields['verified'] = owner['is_verified']
    fields['count_comments'] = node['edge_media_to_comment']['count']
    fields['count_likes'] = node['edge_media_preview_like']['count']
    fields['caption_text'] = None
    fields['has_audio'] = node.get('has_audio')
    fields['comments_disabled'] = node['comments_disabled']
    fields['product_type'] = node.get('product_type')
    fields['gating_info'] = node['gating_info']
    fields['taken_at_timestamp'] = node['taken_at_timestamp']
    fields['edge_liked_by'] = node.get('edge_liked_by', {}).get('count')
    fields['coauthor_producers'] = node.get('coauthor_producers')
    fields['is_professional_account'] = owner['is_professional_account']
    fields['category_name'] = owner['category_name']
    fields['biography'] = owner['biography']
    fields['tagged_user_full_name'] = None
    fields['tagged_is_verified'] = None
    fields['location'] = None 
    fields['edge_media_to_sponsor_user'] = {'edges': []}


    return fields

### Definición de campos

- **_id**: identificador único para cada entrada en el dataset

- **username**: nombre de usuario del influencer en Instagram

- **typename**: tipo de contenido publicado por el influencer (foto, vídeo o multipublicación)

- **is_video**: booleano que indica si la publicación es un video

- **followers**: cantidad de seguidores que tiene el influencer

- **following**: cantidad de cuentas que sigue el influencer

- **verified**: booleano que indica si la cuenta del influencer está verificada por Instagram

- **count_comments**: cantidad de comentarios en la publicación del influencer

- **count_likes**: cantidad de me gustas en la publicación del influencer

- **caption_text**: texto del pie de foto de la publicación

- **tagged_user_full_name**: nombre completo del usuario etiquetado en la publicación

- **tagged_is_verified**: valor que indica si el usuario etiquetado está verificado

- **has_audio**: valor que indica si la publicación tiene audio

- **edge_media_to_sponsor_user**: detalles del usuario patrocinado asociado con la publicación, si los hay

- **comments_disabled**: booleano que indica si los comentarios están deshabilitados para la publicación

- **location**: ubicación geográfica asociada con la publicación

- **product_type**: tipo de producto promovido en la publicación, si lo hay

- **gating_info**: información sobre la publicación, pero esta columna tiene todos sus valores nulos

- **taken_at_timestamp**: marca de tiempo cuando se tomó la foto o el video

- **edge_liked_by**: número de me gustas que recibió la publicación

- **coauthor_producers**: información sobre los coautores o productores de la publicación

- **is_professional_account**: booleano que indica si la cuenta es una cuenta profesional

- **category_name**: categoría de la cuenta o la publicación

- **biography**: biografía del influencer



### Validación

Existen algunas claves del diccionario en las que sus valores asociados pueden venir vacíos o no, es por esto que se repasarán aquellos que en primera instancia fueron grabados con valores **None** para ver si pueden ser rellenados

In [3]:
def verify_field_values(fields, node):
    try: 
        if node['edge_media_to_tagged_user']['edges']:
            fields['tagged_user_full_name'] = node['edge_media_to_tagged_user']['edges'][0]['node']['user']['full_name']
            fields['tagged_is_verified'] = node['edge_media_to_tagged_user']['edges'][0]['node']['user']['is_verified']

        if node['edge_media_to_caption']['edges']:
            fields['caption_text'] = node['edge_media_to_caption']['edges'][0]['node']['text']

        if node['location']:
            fields['location'] = node['location']['name']
     
        if node['edge_media_to_sponsor_user']['edges']:
                fields['edge_media_to_sponsor_user'] = node['edge_media_to_sponsor_user']['edges'][0]['node']['sponsor']['username']
    except:
        pass
    finally: 
        return fields

### Generación de dataframe

Tras haber hecho la primera comprobación de que se ha podido extraer todos los datos que son requeridos para el futuro EDA, el siguiente paso se focaliza **en generar un dataset a partir de la ristra de campos** mencionada en el apartado anterior.

In [4]:
def create_post_df(fields, sector_type, influencer_type):
    fields['sector'] =  sector_type
    fields['influencer_type'] = influencer_type
    
    df = None
    try:
        df = pd.DataFrame(fields, index=[0])
    except:
        pass
    finally:
        return df

El código anterior muestra en primer lugar la adciön de 2 nuevas columnas que permitirán identificar cada post del usuario según el **sector** al que pertenece y como qué **tipo de influencer** es reconocido según los seguidores que posee:

- **Nano-influencer**: [1000, 10000] seguidores
- **Micro-influencer**: (10000, 100000] seguidores
- **Macro-influencer**: 100000, 1000000] seguidores
- **Mega-influencer**: > 1000000 seguidores

### Creación de un dataset a partir de una publicación

Para finalizar este apartado falta un último paso que se basa en la unificación de los distintos procesos descritos anteriormente para obtener un **conjunto de datos que represente a una publicación de Instagram** de un determinado usuario

In [5]:
def generate_dataframe_from_post(post_file, sector_type, influencer_type, fs):    
    # Descargamos y leemos el archivo JSON.xz
    with fs.open(post_file, 'rb') as file:
        json_data = lzma.decompress(file.read())

    # Decodificamos el JSON
    data = json.loads(json_data)

    # Extraemos los campos deseados
    node = data['node']
    owner = node['owner']

    fields = create_field_dict(node, owner)
    # Verificamos si la lista edges tiene al menos un elemento
    fields = verify_field_values(fields, node)
        
    return create_post_df(fields, sector_type, influencer_type)

In [6]:
# Creamos una instancia del sistema de archivos de GCS
fs = gcsfs.GCSFileSystem(project='tfminfluencers')
# Definimos las tuplas que serán necesarias para la recopilación de los datos
influencer_types = ('NANO', 'MICRO', 'MACRO', 'MEGA')
sectors = ('BEAUTY', 'FITNESS')

# Obtenemos la lista de objetos en el bucket "unir_bucket_tfm" dentro de la carpeta "MACRO"
folder_path = 'unir_bucket_tfm/notebooks/jupyter/BEAUTY/MACRO/*/*.json.xz'
file_paths = fs.glob(folder_path)

In [7]:
df = generate_dataframe_from_post(file_paths[0], 'BEAUTY', 'MACRO', fs)

In [8]:
df.head()

,_id,username,typename,is_video,followers,following,verified,count_comments,count_likes,caption_text,...,coauthor_producers,is_professional_account,category_name,biography,tagged_user_full_name,tagged_is_verified,location,edge_media_to_sponsor_user,sector,influencer_type
0,2698249809409637462,aishawari,GraphVideo,True,244628,369,False,71,1997,TATUAJES💉\n\nHoy quería compartir con vosotras...,...,None,True,Digital creator,👩🏼‍💻Tutoriales de maquillaje y recomendaciones...,None,None,None,NaN,BEAUTY,MACRO


## Dataset con todos los influencers de un tipo

Una vez estudiado el proceso con el que poder recopilar un dataframe que tenga en columnas todas aquellas medidas que se desea obtener los ficheros JSON, queda repetir este mismo proceso para todos los ficheros localizados en una misma carpeta y produciendo una tabla para todas las publicaciones existentes en ese directorio.

In [9]:
def create_df_combined(file_paths, sector_type, influencer_type, fs):
    dataframes_list = []
    for post_file in file_paths:
        dataframes_list.append(generate_dataframe_from_post(post_file, sector_type, influencer_type, fs))
    # Concatenamos todos los DataFrames en uno solo
    return pd.concat(dataframes_list, ignore_index=True)

In [10]:
df = create_df_combined(file_paths, 'BEAUTY', 'MACRO', fs)

In [11]:
df

,_id,username,typename,is_video,followers,following,verified,count_comments,count_likes,caption_text,...,coauthor_producers,is_professional_account,category_name,biography,tagged_user_full_name,tagged_is_verified,location,edge_media_to_sponsor_user,sector,influencer_type
0,2698249809409637462,aishawari,GraphVideo,True,244628,369,False,71,1997,TATUAJES💉\n\nHoy quería compartir con vosotras...,...,None,True,Digital creator,👩🏼‍💻Tutoriales de maquillaje y recomendaciones...,None,None,None,NaN,BEAUTY,MACRO
1,2700893032737260603,aishawari,GraphSidecar,False,244628,369,False,69,2994,MAKEUP NO MAKEUP💜\n\nMis maquillajes favoritos...,...,None,True,Digital creator,👩🏼‍💻Tutoriales de maquillaje y recomendaciones...,None,None,None,NaN,BEAUTY,MACRO
2,2704728296303832291,aishawari,GraphSidecar,False,244628,369,False,86,3441,NECESITAS esta blazer y además es de temporada...,...,None,True,Digital creator,👩🏼‍💻Tutoriales de maquillaje y recomendaciones...,ZARA Official,True,"Alicante, Spain",NaN,BEAUTY,MACRO
3,2708213354956317672,aishawari,GraphVideo,True,244628,369,False,83,3780,HAZ ESTO para conseguir una melenita de infart...,...,None,True,Digital creator,👩🏼‍💻Tutoriales de maquillaje y recomendaciones...,ghdspain,True,None,NaN,BEAUTY,MACRO
4,2709868503046925365,aishawari,GraphSidecar,False,244628,369,False,86,1952,CANAS: el proceso continúa\n\nSois muchas las ...,...,None,True,Digital creator,👩🏼‍💻Tutoriales de maquillaje y recomendaciones...,None,None,None,NaN,BEAUTY,MACRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,3074303318055572015,veronica__roman,GraphVideo,True,136616,312,False,3,207,5 tips para elevar tu skincare routine con @id...,...,None,True,"Beauty, cosmetic & personal care",Content Creator \n📍Based in Madrid \nTiktok (+...,IDC Institute,False,"Madrid, Spain",NaN,BEAUTY,MACRO
1650,3079508177665019578,veronica__roman,GraphVideo,True,136616,312,False,4,980,Cute Easy Hairstyle \n\n#hair #hairstyle #hair...,...,None,True,"Beauty, cosmetic & personal care",Content Creator \n📍Based in Madrid \nTiktok (+...,Verónica Román,False,"Madrid, Spain",NaN,BEAUTY,MACRO
1651,3082412845105705618,veronica__roman,GraphSidecar,False,136616,312,False,6,-1,Black soul 🖤,...,None,True,"Beauty, cosmetic & personal care",Content Creator \n📍Based in Madrid \nTiktok (+...,MiiN Cosmetics,True,"Madrid, Spain",NaN,BEAUTY,MACRO
1652,3084442978192809781,veronica__roman,GraphVideo,True,136616,312,False,3,-1,My 7 steps Korean skincare to prepare my skin ...,...,None,True,"Beauty, cosmetic & personal care",Content Creator \n📍Based in Madrid \nTiktok (+...,MiiN Cosmetics,True,"Madrid, Spain",NaN,BEAUTY,MACRO


### Combinación de todos los conjuntos de datos para cualquier sector

In [12]:
def obtains_completed_dataframe():
    fs = gcsfs.GCSFileSystem(project='tfminfluencers')
    # Definimos las tuplas que serán necesarias para la recopilación de los datos
    INFLUENCER_TYPES = ('NANO', 'MICRO', 'MACRO', 'MEGA')
    SECTORS = ('BEAUTY', 'FITNESS')

    FOLDER_PATH = 'unir_bucket_tfm/notebooks/jupyter/' 
    SUFFIX_PATH = '/*/*.json.xz'

    completed_dataframe = None
    df_list = []
    for sector in SECTORS:
        for influencer_type in INFLUENCER_TYPES:
            folder_path = FOLDER_PATH + str(sector) + '/'+ str(influencer_type) + SUFFIX_PATH
            print(folder_path)
            file_paths = fs.glob(folder_path)
            if len(file_paths) > 0:
                df_list.append(create_df_combined(file_paths, sector, influencer_type, fs))
    completed_dataframe = pd.concat(df_list, ignore_index=True)   
    
    return completed_dataframe

In [13]:
completed_combined_df = obtains_completed_dataframe()

unir_bucket_tfm/notebooks/jupyter/BEAUTY/NANO/*/*.json.xz
unir_bucket_tfm/notebooks/jupyter/BEAUTY/MICRO/*/*.json.xz
unir_bucket_tfm/notebooks/jupyter/BEAUTY/MACRO/*/*.json.xz
unir_bucket_tfm/notebooks/jupyter/BEAUTY/MEGA/*/*.json.xz
unir_bucket_tfm/notebooks/jupyter/FITNESS/NANO/*/*.json.xz
unir_bucket_tfm/notebooks/jupyter/FITNESS/MICRO/*/*.json.xz
unir_bucket_tfm/notebooks/jupyter/FITNESS/MACRO/*/*.json.xz
unir_bucket_tfm/notebooks/jupyter/FITNESS/MEGA/*/*.json.xz


In [14]:
completed_combined_df

,_id,username,typename,is_video,followers,following,verified,count_comments,count_likes,caption_text,...,coauthor_producers,is_professional_account,category_name,biography,tagged_user_full_name,tagged_is_verified,location,edge_media_to_sponsor_user,sector,influencer_type
0,1461031291230411282,bazardemarisse,GraphImage,False,1879,1696,False,1,43,Son pendrives aunque no lo parezcan y con ello...,...,None,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Just Cavalli,True,None,NaN,BEAUTY,NANO
1,1461057228236014541,bazardemarisse,GraphImage,False,1879,1696,False,2,49,Hoy hemos conocido un estudio muy interesante ...,...,None,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Cayetana Guillén-Cuervo,True,None,NaN,BEAUTY,NANO
2,1461626070960120583,bazardemarisse,GraphImage,False,1879,1696,False,1,61,Celebrando los 25 años de @uriagespain en siti...,...,None,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Salon des Fleurs,False,Salon des Fleurs,NaN,BEAUTY,NANO
3,1462547320532230958,bazardemarisse,GraphImage,False,1879,1696,False,6,68,Una gama muy amplia y un colorido impresionant...,...,None,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Perfumerías Primor,True,None,NaN,BEAUTY,NANO
4,1463898105857794613,bazardemarisse,GraphImage,False,1879,1696,False,10,76,Las mejores pizzas de Madrid están al lado del...,...,None,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,essie,True,Kilómetros de Pizza,NaN,BEAUTY,NANO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13279,3111938987433797684,vikikacosta,GraphVideo,True,1062361,983,True,59,14286,🦀✅VACACIONES???🙌🏼🎉 Haz este entrenamiento SIN ...,...,None,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",None,None,None,NaN,FITNESS,MEGA
13280,3112105038420076511,vikikacosta,GraphVideo,True,1062361,983,True,131,10653,Irene & Manu 27/05/23 📍Mallorca\nEl vestido de...,...,None,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",PERÒ,False,None,NaN,FITNESS,MEGA
13281,3112645464515939340,vikikacosta,GraphSidecar,False,1062361,983,True,58,11523,"Ayer fue mágico, no lo digo por decir. Irene y...",...,None,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",Javier Menendez I Entrena Virtual #undiamasund...,True,"Mallorca, Islas Baleares, España",NaN,FITNESS,MEGA
13282,3113674384220704610,vikikacosta,GraphVideo,True,1062361,983,True,46,9608,7️⃣👌🏼,...,None,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",None,None,"Mallorca, Islas Baleares, España",NaN,FITNESS,MEGA


## Limpieza de datos

In [17]:
 #!pip install geopy
 #!pip install unidecode
 #!pip install geonamescache

In [18]:
import pandas as pd
import numpy as np
import io
from unidecode import unidecode
import geopy.geocoders
from geonamescache import GeonamesCache
import re

### Análisis del dataset (EDA)

#### Dimensiones

In [19]:
# Se obtiene el total de filas y columnas
print("Hay {} instancias y {} variables en la base de datos.".format(completed_combined_df.shape[0], completed_combined_df.shape[1]))

Hay 13284 instancias y 26 variables en la base de datos.


#### Tipos de variables

- **_id**: identificador único para cada entrada en el dataset

- **username**: nombre de usuario del influencer en Instagram

- **typename**: tipo de contenido publicado por el influencer (foto, vídeo o multipublicación)

- **is_video**: booleano que indica si la publicación es un video

- **followers**: cantidad de seguidores que tiene el influencer

- **following**: cantidad de cuentas que sigue el influencer

- **verified**: booleano que indica si la cuenta del influencer está verificada por Instagram

- **count_comments**: cantidad de comentarios en la publicación del influencer

- **count_likes**: cantidad de me gustas en la publicación del influencer

- **caption_text**: texto del pie de foto de la publicación

- **tagged_user_full_name**: nombre completo del usuario etiquetado en la publicación

- **tagged_is_verified**: valor que indica si el usuario etiquetado está verificado

- **has_audio**: valor que indica si la publicación tiene audio

- **edge_media_to_sponsor_user**: detalles del usuario patrocinado asociado con la publicación, si los hay

- **comments_disabled**: booleano que indica si los comentarios están deshabilitados para la publicación

- **location**: ubicación geográfica asociada con la publicación

- **product_type**: tipo de producto promovido en la publicación, si lo hay

- **gating_info**: información sobre la publicación, pero esta columna tiene todos sus valores nulos

- **taken_at_timestamp**: marca de tiempo cuando se tomó la foto o el video

- **edge_liked_by**: número de me gustas que recibió la publicación

- **coauthor_producers**: información sobre los coautores o productores de la publicación

- **is_professional_account**: booleano que indica si la cuenta es una cuenta profesional

- **category_name**: categoría de la cuenta o la publicación

- **biography**: biografía del influencer

- **influencer_type**: tipo de influencer, puede estar basado en su número de seguidores o su nicho

- **sector**: sector o industria en la que el influencer está más activo

El listado de variables presentado es igual que en la definición de las variables, salvo porque durante el proceso de extracción de los datos a partir del DataLake se añadieron dos nuevas columnas que son **influencer_type y sector**

In [20]:
completed_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13284 entries, 0 to 13283
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   _id                         13284 non-null  object
 1   username                    13284 non-null  object
 2   typename                    13284 non-null  object
 3   is_video                    13284 non-null  bool  
 4   followers                   13284 non-null  int64 
 5   following                   13284 non-null  int64 
 6   verified                    13284 non-null  bool  
 7   count_comments              13284 non-null  int64 
 8   count_likes                 13284 non-null  int64 
 9   caption_text                13226 non-null  object
 10  has_audio                   6071 non-null   object
 11  comments_disabled           13284 non-null  bool  
 12  product_type                6071 non-null   object
 13  gating_info                 0 non-null      ob

Como pandas ha clasificado muchas de las variables como "object" vamos a ver qué tipo de información contienen para poder asignarles un tipo mejor si procede.

In [21]:
completed_combined_df.username.value_counts()

diegoo.davila           198
pau_eche                185
santiliebanatraining    184
mariapicazodn           174
konayasakimori          173
                       ... 
vikikacosta              65
susana_bicho             54
clarasbeautyshelf        38
violeta                  10
fitconmaria               4
Name: username, Length: 96, dtype: int64

In [22]:
completed_combined_df.typename.value_counts()

GraphVideo      6071
GraphSidecar    4066
GraphImage      3147
Name: typename, dtype: int64

La variable **'typename' es catégorica**. Luego le cambiaremos el tipo de forma acorde.

In [23]:
completed_combined_df.caption_text.value_counts()

#Motivation #followme #goals #workout #fitnessaddict #beastmode #biceps #muscles #healthyfood #shoulder #iffbpro #goodmorning #relaxedvsflexed #bodybuildingmotivation #gainz #menphysique #physique #physics #oldschool #instasize  #shredder #goodmorning                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     54
#aesthetic #Motivation #followme #goa

In [24]:
completed_combined_df.tagged_user_full_name.value_counts()

Gym Virtual✌🏻✊🏻🖐🏻                206
Perfumerías Primor              110
                                 101
HUDA BEAUTY                       99
Reme Navarro Skin&Nutrition       96
                                ... 
The Essentials Cosmetics           1
Schiaparelli                       1
e.l.f. Cosmetics and Skincare      1
We Make-up, and you?               1
FERENT BAGS                        1
Name: tagged_user_full_name, Length: 1710, dtype: int64

In [25]:
completed_combined_df.tagged_is_verified.value_counts()

True     4314
False    2105
Name: tagged_is_verified, dtype: int64

La variable **'tagged_is_verified' es booleana**.

In [26]:
completed_combined_df.influencer_type.value_counts()

MACRO    3333
MICRO    3332
MEGA     3310
NANO     3309
Name: influencer_type, dtype: int64

La variable **'influencer_type' es catégorica**.

In [27]:
completed_combined_df.sector.value_counts()

FITNESS    6685
BEAUTY     6599
Name: sector, dtype: int64

In [28]:
completed_combined_df.has_audio.value_counts()

True     6011
False      60
Name: has_audio, dtype: int64

La variable **'has_audio' es booleana.**

In [29]:
completed_combined_df.edge_media_to_sponsor_user.value_counts()

maxfactores            12
saigucosmetics         12
industrialbeautyes     10
nablacosmetics          9
kiehlsspain             8
                       ..
cetaphil.es             1
freshlycosmetics_es     1
justspices_es           1
amo.como.soy            1
revlon_es               1
Name: edge_media_to_sponsor_user, Length: 84, dtype: int64

In [30]:
completed_combined_df.location.value_counts()

Madrid, Spain                449
Barcelona, Spain             268
Valencia                     231
Tenerife                     118
Zaragoza, Spain              111
                            ... 
Masca, Canarias, Spain         1
Rio Guadalquivir, Sevilla      1
Girona                         1
IKEA                           1
Novotel Madrid Center          1
Name: location, Length: 890, dtype: int64

In [31]:
completed_combined_df.product_type.value_counts()

clips    5641
igtv      264
feed      166
Name: product_type, dtype: int64

La variable **'product_type' es categórica**.

In [32]:
completed_combined_df.coauthor_producers.value_counts()

Series([], Name: coauthor_producers, dtype: int64)

Esta variable trae siempre una lista vacía [], posteriormente en la homogeneización lo pondremos como nulo.

In [33]:
completed_combined_df.category_name.value_counts()

Digital creator                          1561
Blogger                                  1156
Personal blog                            1141
Beauty, cosmetic & personal care         1133
Coach                                    1129
Public figure                            1077
Fitness Trainer                           905
Athlete                                   901
Entrepreneur                              628
Health/beauty                             612
Nutritionist                              543
Makeup Artist                             386
Community                                 331
Artist                                    306
Fitness Model                             268
Actor                                     170
Sports                                    142
Scientist                                 139
Journalist                                138
Alternative & Holistic Health Service     109
Name: category_name, dtype: int64

La variable **'category_name' es categórica**.

In [34]:
completed_combined_df.biography.value_counts()

🔥Ayudo a personas delgadas de 20-40 años que entrenan y no ven resultados a ganar +5kg en 12 semanas🔥\nEscríbeme la palabra “CAMBIO” y te ayudo ⬇️         198
ᴄʜᴀɴɢᴇ ɪs ᴛʜᴇ ᴏɴʟʏ ᴄᴏɴsᴛᴀɴᴛ... 😘🇪🇸     \nContacto: ana@anatenorio.com\nTikTok @pau_eche                                                                    185
Profesor de cursos de formación y entrenador desde 2002. Resumen de vainas en el LinkTree.                                                                 184
🎓Sport&Coaching Nutrition\n🧠Cambiemos hábitos y actitud\n🏡 @centro.mariapicazo @dassportclinic \n⚽️ Responsable de Cantera @cdtoficial @cdtfundacion ⠀⠀    174
BREATHING FLOW\nPreparador físico / Readaptador\nNeurología aplicada y Respiración\nBJJ Mathias Ribeiro Team 🟦\nJudo AJM 🟦                                 173
                                                                                                                                                          ... 
TikTok:violeta                                

In [35]:
# Cambiamos el tipo de las variables que hemos visto antes
# creamos una lista con todas las variables de tipo object que vamos a convertir
target_cols = ['typename', 'influencer_type', 'sector', 'product_type','category_name']
cols_to_categoric = df[target_cols].columns.tolist()
cols_to_categoric

['typename', 'influencer_type', 'sector', 'product_type', 'category_name']

In [36]:
# Convertimos todas esas variables de golpe:
completed_combined_df[cols_to_categoric] = completed_combined_df[cols_to_categoric].astype("category")

In [37]:
completed_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13284 entries, 0 to 13283
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   _id                         13284 non-null  object  
 1   username                    13284 non-null  object  
 2   typename                    13284 non-null  category
 3   is_video                    13284 non-null  bool    
 4   followers                   13284 non-null  int64   
 5   following                   13284 non-null  int64   
 6   verified                    13284 non-null  bool    
 7   count_comments              13284 non-null  int64   
 8   count_likes                 13284 non-null  int64   
 9   caption_text                13226 non-null  object  
 10  has_audio                   6071 non-null   object  
 11  comments_disabled           13284 non-null  bool    
 12  product_type                6071 non-null   category
 13  gating_info     

In [38]:
# Cambiamos el tipo de las variables que hemos visto antes
# creamos una lista con todas las variables de tipo object que vamos a convertir
target_cols = ['tagged_is_verified', 'has_audio']
cols_to_boolean = df[target_cols].columns.tolist()
cols_to_boolean

['tagged_is_verified', 'has_audio']

In [39]:
# Convertimos todas esas variables de golpe:
completed_combined_df[cols_to_boolean] = completed_combined_df[cols_to_boolean].astype("bool")

In [40]:
completed_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13284 entries, 0 to 13283
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   _id                         13284 non-null  object  
 1   username                    13284 non-null  object  
 2   typename                    13284 non-null  category
 3   is_video                    13284 non-null  bool    
 4   followers                   13284 non-null  int64   
 5   following                   13284 non-null  int64   
 6   verified                    13284 non-null  bool    
 7   count_comments              13284 non-null  int64   
 8   count_likes                 13284 non-null  int64   
 9   caption_text                13226 non-null  object  
 10  has_audio                   13284 non-null  bool    
 11  comments_disabled           13284 non-null  bool    
 12  product_type                6071 non-null   category
 13  gating_info     

### Estandarización

#### Formato fecha y hora

La única columna que tiene datos de fecha y hora es la de **taken_at_timestamp**.

In [41]:
completed_combined_df['taken_at_timestamp'].head()

0    1488388523
1    1488391615
2    1488459427
3    1488569248
4    1488730274
Name: taken_at_timestamp, dtype: int64

Estos numeros son marcas de tiempo en formato UNIX, que es el número de segundos que han transcurrido desde la medianoche UTC del 1 de enero de 1970, sin contar los segundos de los años bisiestos. Por lo tanto, lo convertimos esto al formato estándar internacional de referencia ISO 8601.

In [42]:
completed_combined_df['taken_at_timestamp'] = pd.to_datetime(completed_combined_df['taken_at_timestamp'], unit='s')

Mostramos el nuevo formato de la columna

In [43]:
completed_combined_df['taken_at_timestamp'].head()

0   2017-03-01 17:15:23
1   2017-03-01 18:06:55
2   2017-03-02 12:57:07
3   2017-03-03 19:27:28
4   2017-03-05 16:11:14
Name: taken_at_timestamp, dtype: datetime64[ns]

### Atomización de valores

Se considera que hay valores "no atómicos" cuando en un campo hay más de 1 único valor/concepto.

#### Location

In [44]:
completed_combined_df.location.value_counts(dropna=False)

NaN                          9680
Madrid, Spain                 449
Barcelona, Spain              268
Valencia                      231
Tenerife                      118
                             ... 
Puente Del Rio Tinto            1
Kilómetros de Pizza             1
Abu-Dabi                        1
Velazquez 12                    1
Algún Lugar De La Galaxia       1
Name: location, Length: 891, dtype: int64

In [45]:
geolocator = geopy.geocoders.Nominatim(user_agent='my_geocoder')
geonames = GeonamesCache()

def geolocation(row):
    if pd.notnull(row['location']):
        location = row['location'].strip()
        try:
            geocode = geolocator.geocode(location)
            if geocode:
                address_parts = geocode.address.split(', ')
                if len(address_parts) >= 2:
                    return address_parts[-2], address_parts[-1]
        except geopy.exc.GeocoderTimedOut:
            return None
        except geopy.exc.GeocoderUnavailable:
            return None
    return None

def city_assigning(row):
    if pd.notnull(row['geolocation']):
        city_code = row['geolocation'][0]
        city_data = geonames.get_cities().get(city_code)
        if city_data:
            return city_data['name']
    return 'N/A'

def country_assigning(row):
    if pd.notnull(row['geolocation']):
        return row['geolocation'][1]
    return 'N/A'

In [46]:
# Aplicamos la función de geolocalización a la columna 'location' y creamos una nueva columna 'geolocation'
# completed_combined_df['geolocation'] = completed_combined_df.apply(geolocation, axis=1)

In [47]:
len(completed_combined_df.index)   

13284

Es necesario separar las llamadas a la librería geopy porque de lo contrario devuelve un error al intentar hacer demasiadas peticiones en un mismo proceso

In [48]:
split_size = 500
sub_df_list = []
row_number_altered = None
for i in range(1,6):
    row_number_altered = split_size * i
    sub_df_list.append(completed_combined_df.iloc[row_number_altered - split_size:row_number_altered,:].apply(geolocation, axis=1))
    print("Dataframe " + str(i) + " geolocalizado")

Dataframe 1 geolocalizado
Dataframe 2 geolocalizado
Dataframe 3 geolocalizado
Dataframe 4 geolocalizado
Dataframe 5 geolocalizado


In [49]:
for i in range(6,12):
    row_number_altered = split_size * i
    sub_df_list.append(completed_combined_df.iloc[row_number_altered - split_size:row_number_altered,:].apply(geolocation, axis=1))
    print("Dataframe " + str(i) + " geolocalizado")


Dataframe 6 geolocalizado
Dataframe 7 geolocalizado
Dataframe 8 geolocalizado
Dataframe 9 geolocalizado
Dataframe 10 geolocalizado
Dataframe 11 geolocalizado


In [ ]:
for i in range(12,18):
    row_number_altered = split_size * i
    sub_df_list.append(completed_combined_df.iloc[row_number_altered - split_size:row_number_altered,:].apply(geolocation, axis=1))
    print("Dataframe " + str(i) + " geolocalizado")


Dataframe 12 geolocalizado
Dataframe 13 geolocalizado
Dataframe 14 geolocalizado
Dataframe 15 geolocalizado
Dataframe 16 geolocalizado
Dataframe 17 geolocalizado


In [51]:
for i in range(18,23):
    row_number_altered = split_size * i
    sub_df_list.append(completed_combined_df.iloc[row_number_altered - split_size:row_number_altered,:].apply(geolocation, axis=1))
    print("Dataframe " + str(i) + " geolocalizado")


Dataframe 18 geolocalizado
Dataframe 19 geolocalizado
Dataframe 20 geolocalizado
Dataframe 21 geolocalizado
Dataframe 22 geolocalizado


In [52]:
for i in range(23,26):
    row_number_altered = split_size * i
    sub_df_list.append(completed_combined_df.iloc[row_number_altered - split_size:row_number_altered,:].apply(geolocation, axis=1))
    print("Dataframe " + str(i) + " geolocalizado")
sub_df_list.append(completed_combined_df.iloc[row_number_altered:,:].apply(geolocation, axis=1))


Dataframe 23 geolocalizado
Dataframe 24 geolocalizado
Dataframe 25 geolocalizado


In [53]:
sub_df_combined = pd.concat(sub_df_list, ignore_index=True)

In [54]:
len(sub_df_combined)

13284

In [55]:
sub_df_combined

0                           None
1                           None
2                (116 35, Ελλάς)
3                           None
4                (28050, España)
                  ...           
13279                       None
13280                       None
13281    (Illes Balears, España)
13282    (Illes Balears, España)
13283                       None
Length: 13284, dtype: object

In [56]:
completed_combined_df['geolocation'] = sub_df_combined

In [57]:
completed_combined_df

,_id,username,typename,is_video,followers,following,verified,count_comments,count_likes,caption_text,...,is_professional_account,category_name,biography,tagged_user_full_name,tagged_is_verified,location,edge_media_to_sponsor_user,sector,influencer_type,geolocation
0,1461031291230411282,bazardemarisse,GraphImage,False,1879,1696,False,1,43,Son pendrives aunque no lo parezcan y con ello...,...,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Just Cavalli,True,None,NaN,BEAUTY,NANO,None
1,1461057228236014541,bazardemarisse,GraphImage,False,1879,1696,False,2,49,Hoy hemos conocido un estudio muy interesante ...,...,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Cayetana Guillén-Cuervo,True,None,NaN,BEAUTY,NANO,None
2,1461626070960120583,bazardemarisse,GraphImage,False,1879,1696,False,1,61,Celebrando los 25 años de @uriagespain en siti...,...,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Salon des Fleurs,False,Salon des Fleurs,NaN,BEAUTY,NANO,"(116 35, Ελλάς)"
3,1462547320532230958,bazardemarisse,GraphImage,False,1879,1696,False,6,68,Una gama muy amplia y un colorido impresionant...,...,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Perfumerías Primor,True,None,NaN,BEAUTY,NANO,None
4,1463898105857794613,bazardemarisse,GraphImage,False,1879,1696,False,10,76,Las mejores pizzas de Madrid están al lado del...,...,True,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,essie,True,Kilómetros de Pizza,NaN,BEAUTY,NANO,"(28050, España)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13279,3111938987433797684,vikikacosta,GraphVideo,True,1062361,983,True,59,14286,🦀✅VACACIONES???🙌🏼🎉 Haz este entrenamiento SIN ...,...,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",None,False,None,NaN,FITNESS,MEGA,None
13280,3112105038420076511,vikikacosta,GraphVideo,True,1062361,983,True,131,10653,Irene & Manu 27/05/23 📍Mallorca\nEl vestido de...,...,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",PERÒ,False,None,NaN,FITNESS,MEGA,None
13281,3112645464515939340,vikikacosta,GraphSidecar,False,1062361,983,True,58,11523,"Ayer fue mágico, no lo digo por decir. Irene y...",...,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",Javier Menendez I Entrena Virtual #undiamasund...,True,"Mallorca, Islas Baleares, España",NaN,FITNESS,MEGA,"(Illes Balears, España)"
13282,3113674384220704610,vikikacosta,GraphVideo,True,1062361,983,True,46,9608,7️⃣👌🏼,...,True,Entrepreneur,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",None,False,"Mallorca, Islas Baleares, España",NaN,FITNESS,MEGA,"(Illes Balears, España)"


In [58]:
# Creamos los campos 'city' y 'country' a partir de la columna 'geolocation' y clasificamos los no asignados como "N/A"
completed_combined_df['city'] = completed_combined_df.apply(city_assigning, axis=1)
completed_combined_df['country'] = completed_combined_df.apply(country_assigning, axis=1)

In [59]:
completed_combined_df

,_id,username,typename,is_video,followers,following,verified,count_comments,count_likes,caption_text,...,biography,tagged_user_full_name,tagged_is_verified,location,edge_media_to_sponsor_user,sector,influencer_type,geolocation,city,country
0,1461031291230411282,bazardemarisse,GraphImage,False,1879,1696,False,1,43,Son pendrives aunque no lo parezcan y con ello...,...,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Just Cavalli,True,None,NaN,BEAUTY,NANO,None,N/A,N/A
1,1461057228236014541,bazardemarisse,GraphImage,False,1879,1696,False,2,49,Hoy hemos conocido un estudio muy interesante ...,...,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Cayetana Guillén-Cuervo,True,None,NaN,BEAUTY,NANO,None,N/A,N/A
2,1461626070960120583,bazardemarisse,GraphImage,False,1879,1696,False,1,61,Celebrando los 25 años de @uriagespain en siti...,...,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Salon des Fleurs,False,Salon des Fleurs,NaN,BEAUTY,NANO,"(116 35, Ελλάς)",N/A,Ελλάς
3,1462547320532230958,bazardemarisse,GraphImage,False,1879,1696,False,6,68,Una gama muy amplia y un colorido impresionant...,...,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Perfumerías Primor,True,None,NaN,BEAUTY,NANO,None,N/A,N/A
4,1463898105857794613,bazardemarisse,GraphImage,False,1879,1696,False,10,76,Las mejores pizzas de Madrid están al lado del...,...,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,essie,True,Kilómetros de Pizza,NaN,BEAUTY,NANO,"(28050, España)",N/A,España
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13279,3111938987433797684,vikikacosta,GraphVideo,True,1062361,983,True,59,14286,🦀✅VACACIONES???🙌🏼🎉 Haz este entrenamiento SIN ...,...,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",None,False,None,NaN,FITNESS,MEGA,None,N/A,N/A
13280,3112105038420076511,vikikacosta,GraphVideo,True,1062361,983,True,131,10653,Irene & Manu 27/05/23 📍Mallorca\nEl vestido de...,...,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",PERÒ,False,None,NaN,FITNESS,MEGA,None,N/A,N/A
13281,3112645464515939340,vikikacosta,GraphSidecar,False,1062361,983,True,58,11523,"Ayer fue mágico, no lo digo por decir. Irene y...",...,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",Javier Menendez I Entrena Virtual #undiamasund...,True,"Mallorca, Islas Baleares, España",NaN,FITNESS,MEGA,"(Illes Balears, España)",N/A,España
13282,3113674384220704610,vikikacosta,GraphVideo,True,1062361,983,True,46,9608,7️⃣👌🏼,...,"🌟Ponte metas, no límites. \n@entrenavirtual.es...",None,False,"Mallorca, Islas Baleares, España",NaN,FITNESS,MEGA,"(Illes Balears, España)",N/A,España


#### taken_at_timestamp

Este campo contiene información tanto de la fecha como de la hora. Lo dividimos en 2 campos: Fecha y Hora.

In [60]:
# Convertimos la columna 'taken_at_timestamp' a tipo cadena
completed_combined_df['taken_at_timestamp'] = completed_combined_df['taken_at_timestamp'].astype(str)

# Dividimos la columna 'taken_at_timestamp' en 'Fecha' y 'Hora'
completed_combined_df[['Date', 'Hour']] = completed_combined_df['taken_at_timestamp'].str.split(' ', 1, expand=True)

# Mostramos el resultado
print(completed_combined_df[['Date', 'Hour']].head())

         Date      Hour
0  2017-03-01  17:15:23
1  2017-03-01  18:06:55
2  2017-03-02  12:57:07
3  2017-03-03  19:27:28
4  2017-03-05  16:11:14


#### coauthor_producers

In [61]:
# En la sección anterior de EDA se ha visto que el campo "coauthor_producers" viene siempre informado con una lista vacía [].
# No obstante, hay otros campos que cuando no hay información aparece NaN.
# Cambiamos por tanto el valor "[]" por NaN para dicho campo.
completed_combined_df['coauthor_producers'] = completed_combined_df['coauthor_producers'].apply(lambda x: np.NaN if x == '[]' else x)
completed_combined_df.coauthor_producers.value_counts()

Series([], Name: coauthor_producers, dtype: int64)

In [62]:
# verificamos que el cambio se ha realizado correctamente
print(completed_combined_df['coauthor_producers'].head())

0    None
1    None
2    None
3    None
4    None
Name: coauthor_producers, dtype: object


### Valores nulos

Para entrar en contexto, se vuelven a imprimir el número de filas que hay en total

In [63]:
len(completed_combined_df.index)

13284

Se cuentan cuántos valores nulos hay en cada columna

In [83]:
completed_combined_df.isnull().sum()

_id                            0
username                       0
typename                       0
followers                      0
following                      0
verified                       0
count_comments                 0
count_likes                    0
caption_text                   0
has_audio                      0
comments_disabled              0
product_type                7213
taken_at_timestamp             0
edge_liked_by              13284
is_professional_account        0
category_name                509
biography                      0
tagged_user_full_name          0
tagged_is_verified             0
location                    9680
sector                         0
influencer_type                0
geolocation                10304
country                        0
Date                           0
Hour                           0
dtype: int64

Se calcula el porcentaje presente en cada variable

In [65]:
null_percentage = completed_combined_df.isnull().sum() / len(completed_combined_df) * 100
print(null_percentage)

_id                             0.000000
username                        0.000000
typename                        0.000000
is_video                        0.000000
followers                       0.000000
following                       0.000000
verified                        0.000000
count_comments                  0.000000
count_likes                     0.000000
caption_text                    0.436615
has_audio                       0.000000
comments_disabled               0.000000
product_type                   54.298404
gating_info                   100.000000
taken_at_timestamp              0.000000
edge_liked_by                 100.000000
coauthor_producers            100.000000
is_professional_account         0.000000
category_name                   3.831677
biography                       0.000000
tagged_user_full_name          51.678711
tagged_is_verified              0.000000
location                       72.869618
edge_media_to_sponsor_user     98.562180
sector          

#### Caso 1) Tratamiento de valores nulos: eliminación de campos

Los campos 'gating_info','edge_liked_by' y 'coauthor_producers' tienen todos sus valores nulos al 100% y el campo 'edge_media_to_sponsor_user' tiene el 98,5% de los valores en nulo por lo que se eliminan ya que no aportan información. Además, se vio anteriormente que sucedía lo mismo con el campo 'city' ya que tiene todos los valores en N/A y no aporta información de valor.

In [66]:
#vemos los valores que contiene 'city'
completed_combined_df.value_counts('city')

city
N/A    13284
dtype: int64

In [68]:
# Eliminamos las siguientes columnas ya que no aportan información
completed_combined_df = completed_combined_df.drop('gating_info', axis=1)
completed_combined_df = completed_combined_df.drop('edge_liked_by', axis=1)
completed_combined_df = completed_combined_df.drop('edge_media_to_sponsor_user', axis=1)
completed_combined_df = completed_combined_df.drop('coauthor_producers', axis=1)
completed_combined_df = completed_combined_df.drop('city', axis=1)
completed_combined_df.info()

KeyError: "['gating_info'] not found in axis"

#### Caso 2) Tratamiento de valores nulos: asignación de valores

Existen campos para los que el valor nulo, por propia constitución del json, no significa ausencia de datos sino 0 o ninguno. Por ello, adaptamos el valor de dichos campos.

##### tagged_user_full_name

In [69]:
#vemos los valores que contiene
completed_combined_df.value_counts('tagged_user_full_name')

tagged_user_full_name
Gym Virtual✌🏻✊🏻🖐🏻                  206
Perfumerías Primor                110
                                   101
HUDA BEAUTY                         99
Reme Navarro Skin&Nutrition         96
                                  ... 
MotoGP™                              1
Dra.Alexia | Medicina Estética      1
Mounir                               1
Dra. Natalia Gennaro                 1
🧿𝗔𝗿𝘁𝘂𝗿𝗼 Á𝗹𝘃𝗮𝗿𝗲𝘇-𝗕𝗮𝘂𝘁𝗶𝘀𝘁𝗮🧿           1
Length: 1710, dtype: int64

In [70]:
# Sustituimos los valores nulos por un 'None' ya que significa que no hay ningún usuario tageado en esa publicación.
completed_combined_df['tagged_user_full_name'] = completed_combined_df['tagged_user_full_name'].fillna('None')

In [71]:
# Comprobamos que ahora ese campo no contiene valores nulos
completed_combined_df.isnull().sum()

_id                            0
username                       0
typename                       0
is_video                       0
followers                      0
following                      0
verified                       0
count_comments                 0
count_likes                    0
caption_text                  58
has_audio                      0
comments_disabled              0
product_type                7213
taken_at_timestamp             0
edge_liked_by              13284
is_professional_account        0
category_name                509
biography                      0
tagged_user_full_name          0
tagged_is_verified             0
location                    9680
sector                         0
influencer_type                0
geolocation                10304
country                        0
Date                           0
Hour                           0
dtype: int64

##### Campo 'caption_text'

Lo mismo para el campo 'caption_text'

In [73]:
# Sustituimos los valores nulos por un 'None' ya que significa que no hay ningún usuario tageado en esa publicación.
completed_combined_df['caption_text'] = completed_combined_df['caption_text'].fillna('None')

In [74]:
completed_combined_df.isnull().sum()

_id                            0
username                       0
typename                       0
is_video                       0
followers                      0
following                      0
verified                       0
count_comments                 0
count_likes                    0
caption_text                   0
has_audio                      0
comments_disabled              0
product_type                7213
taken_at_timestamp             0
edge_liked_by              13284
is_professional_account        0
category_name                509
biography                      0
tagged_user_full_name          0
tagged_is_verified             0
location                    9680
sector                         0
influencer_type                0
geolocation                10304
country                        0
Date                           0
Hour                           0
dtype: int64

En el resto de campos que contienen nulos sí que se entiende que el nulo significa ausencia de datos por lo que no conviene cambiarles el valor.

Por otro lado, se cree conveniente no eliminar ningún registro que pueda contener algún campo sin información ya que puede tener otros campos que sí vengan informados y sean útiles de cara al análisis.

### Valores duplicados

In [75]:
completed_combined_df.duplicated().sum()

0

### Valores redundantes

#### Campos 'typename' y 'is_video'

Este problema sucede cuando una misma información se proporciona en diferentes variables.

La variable **'typename' indica si es una imagen, video o multipublicación**, por lo que la variable 'is_video' sería redundante, ya que ya nos lo estaría diciendo ya la variable anterior

Primero vamos a ver los valores únicos de cada una para comprobar si realmente tienen los mismos valores. Para ello cuando la variable 'is_video' es 'True', la variable 'typename' debería ser 'GraphVideo'

In [76]:
completed_combined_df['is_video'].value_counts()

False    7213
True     6071
Name: is_video, dtype: int64

In [77]:
completed_combined_df['typename'].value_counts()

GraphVideo      6071
GraphSidecar    4066
GraphImage      3147
Name: typename, dtype: int64

Como hemos podido comprobar hay 6071 que tienen valor 'True' en la variable 'is_video' y 6071 'typename' con valor 'GraphVideo'. Por lo tanto, podríamos prescindir de la variable 'is_video', que nos da mucha menos información que la otra.

In [78]:
completed_combined_df = completed_combined_df.drop('is_video', axis=1)

#### Campos 'location', 'geolocalizacion' y 'pais'

Las variables 'location' y 'geolocalizacion' aportan información no atómica y redundante ya que el campo 'country' se ha creado a partir de éstas. Podemos, por tanto, prescindir de ellas.

In [90]:
completed_combined_df['geolocation'].value_counts()

(Comunidad de Madrid, España)     605
(08001, España)                   294
(Comunitat Valenciana, España)    275
(Canarias, España)                273
(Andalucía, España)               192
                                 ... 
(województwo śląskie, Polska)       1
(19420, España)                     1
(80363, Indonesia)                  1
(35560, France)                     1
(11500, España)                     1
Name: geolocation, Length: 406, dtype: int64

In [91]:
completed_combined_df['location'].value_counts()

Madrid, Spain                449
Barcelona, Spain             268
Valencia                     231
Tenerife                     118
Zaragoza, Spain              111
                            ... 
Masca, Canarias, Spain         1
Rio Guadalquivir, Sevilla      1
Girona                         1
IKEA                           1
Novotel Madrid Center          1
Name: location, Length: 890, dtype: int64

In [92]:
completed_combined_df['country'].value_counts()

N/A                               10304
España                             2306
United States                       143
الإمارات العربية المتحدة            104
France                               83
México                               51
Italia                               40
United Kingdom                       40
日本                                   23
ދިވެހިރާއްޖެ                         21
Andorra                              17
Deutschland                          15
Portugal                             13
Brasil                               12
Maroc / ⵍⵎⵖⵔⵉⴱ / المغرب               8
Lietuva                               6
ประเทศไทย                             6
Türkiye                               5
Argentina                             5
Ελλάς                                 4
Ecuador                               4
Nederland                             4
India                                 4
Россия                                4
Indonesia                             4


In [93]:
completed_combined_df = completed_combined_df.drop('geolocation', axis=1)
completed_combined_df = completed_combined_df.drop('location', axis=1)

#### Campos 'take_at_timestamp' y 'Date' y 'Hour'

In [94]:
completed_combined_df.groupby("taken_at_timestamp")["Date"].unique().head(5)

taken_at_timestamp
2016-12-05 12:29:23    [2016-12-05]
2016-12-14 17:37:15    [2016-12-14]
2016-12-17 14:33:27    [2016-12-17]
2017-01-14 13:52:41    [2017-01-14]
2017-01-23 09:03:24    [2017-01-23]
Name: Date, dtype: object

In [95]:
completed_combined_df.groupby("taken_at_timestamp")["Hour"].unique().head(5)

taken_at_timestamp
2016-12-05 12:29:23    [12:29:23]
2016-12-14 17:37:15    [17:37:15]
2016-12-17 14:33:27    [14:33:27]
2017-01-14 13:52:41    [13:52:41]
2017-01-23 09:03:24    [09:03:24]
Name: Hour, dtype: object

Los campos 'Date' y 'Hour' fueron creados anteriormente a partir del campo 'taken_at_timestamp'. Como el campo 'taken_at_timestamp' proporciona ahora información redundante, se elimina del dataset.

### Renombrar variables

Los nombres de las columnas actuales son los siguientes:

In [96]:
column_names = completed_combined_df.columns.tolist()
column_names

['_id',
 'username',
 'typename',
 'followers',
 'following',
 'verified',
 'count_comments',
 'count_likes',
 'caption_text',
 'has_audio',
 'comments_disabled',
 'product_type',
 'taken_at_timestamp',
 'is_professional_account',
 'category_name',
 'biography',
 'tagged_user_full_name',
 'tagged_is_verified',
 'sector',
 'influencer_type',
 'country',
 'Date',
 'Hour']

La mayoría de los nombres de las variables son intuitivos y descriptivos, pero se renombran las siguientes variables para facilitar la interpretabilidad:

In [98]:
completed_combined_df = completed_combined_df.rename(columns={'product_type': 'post_type'})
completed_combined_df = completed_combined_df.rename(columns={'taken_at_timestamp': 'timestamp'})

In [99]:
new_column_names = completed_combined_df.columns.tolist()
new_column_names

['_id',
 'username',
 'typename',
 'followers',
 'following',
 'verified',
 'count_comments',
 'count_likes',
 'caption_text',
 'has_audio',
 'comments_disabled',
 'post_type',
 'timestamp',
 'is_professional_account',
 'category_name',
 'biography',
 'tagged_user_full_name',
 'tagged_is_verified',
 'sector',
 'influencer_type',
 'country',
 'Date',
 'Hour']

### Variables nuevas

#### Hastags

In [100]:
# Definimos una función para extraer los hashtags de un texto
def extract_hastags(text):
    hashtags = re.findall(r'#(\w+)', text)
    return hashtags

# Aplicamos la función a la columna 'caption_text' y creamos el nuevo campo 'hashtags'
completed_combined_df['hashtags'] = completed_combined_df['caption_text'].apply(extract_hastags)

# Convertimos las listas en filas separadas
df_exploded = completed_combined_df.explode('hashtags')


In [101]:
# Obtener los valores únicos de 'hashtags' agrupados por 'caption_text'
completed_combined_df.head(5)

,_id,username,typename,followers,following,verified,count_comments,count_likes,caption_text,has_audio,...,category_name,biography,tagged_user_full_name,tagged_is_verified,sector,influencer_type,country,Date,Hour,hashtags
0,1461031291230411282,bazardemarisse,GraphImage,1879,1696,False,1,43,Son pendrives aunque no lo parezcan y con ello...,False,...,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Just Cavalli,True,BEAUTY,NANO,N/A,2017-03-01,17:15:23,"[guerradependrives, pendrives, beautyblog, fas..."
1,1461057228236014541,bazardemarisse,GraphImage,1879,1696,False,2,49,Hoy hemos conocido un estudio muy interesante ...,False,...,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Cayetana Guillén-Cuervo,True,BEAUTY,NANO,N/A,2017-03-01,18:06:55,"[blogdebelleza, beautyblog, meatrevoaseryo, ni..."
2,1461626070960120583,bazardemarisse,GraphImage,1879,1696,False,1,61,Celebrando los 25 años de @uriagespain en siti...,False,...,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Salon des Fleurs,False,BEAUTY,NANO,Ελλάς,2017-03-02,12:57:07,"[beautyblog, blogdebelleza, uriage, felizcumpl..."
3,1462547320532230958,bazardemarisse,GraphImage,1879,1696,False,6,68,Una gama muy amplia y un colorido impresionant...,False,...,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,Perfumerías Primor,True,BEAUTY,NANO,N/A,2017-03-03,19:27:28,"[beautyblog, blogdebelleza, bronxcolors, makeu..."
4,1463898105857794613,bazardemarisse,GraphImage,1879,1696,False,10,76,Las mejores pizzas de Madrid están al lado del...,False,...,Personal blog,💄 Beauty 👗 Fashion and 👯‍♀️ Lifestyle blogger\...,essie,True,BEAUTY,NANO,España,2017-03-05,16:11:14,"[lifestyleblog, lifestyle, restaurantemadrid, ..."


#### Hashtags Beauty

In [137]:
# Definimos la lista de palabras clave del sector belleza

keywords = ["makeup", "maquilla", "contour", "contorn", "bronzer", "glitter", "lip", "eyeliner", "cejas", "browlift", "beauty", "cosmetic",
                  "dermatolog", "pestañas", "lashes", "belleza", "skin", "rutin", "piel", "pelo", "uñas", "nail", "antiedad", "antiag", "face",
                  "protectorsolar", "sunscreen", "cuidado", "cabello", "haircare", "ojeras", "darkcircles", "gloss", "glow", "ceramidecapsules",
                  "ad", "publi", "colaboracion", "EsteePartner", "proteccionsolar", "natural", "motd", "clean", "care", "crueltyfree", "glam",
                  "essential", "holistic", "byme", "ritual", "pampering", "kbeauty", "bblogger", "auronic", "serum", "aloevera", "crem", "estria"
                  "gel","bronceador", "brill", "labios", "delineador", "levantadocejas", "luz", "limpi", "manicur", "pedicur", "rostro", "facial",
                  "estetic", "hidrata", "hydrat", "tonific", "exfoli", "mask", "mascarilla", "tonic", "arrugas", "manchas", "acne", "gran", "rosac",
                  "puntosnegro", "poros", "rosacea", "sensibilidad", "flacidez", "celuli", "tone", "wrinkles","spots", "pimple","blackhead", "pores",
                  "sensitivity", "sagging",  "cellulit", "stretchmarks"]


def verify_hastags(row):
    if row['sector'] == 'BEAUTY':
        for keyword in keywords:
            if any(re.search(rf"{keyword}", hashtag, re.IGNORECASE) for hashtag in row['hashtags']):
                return 'Y'
        else:
            return 'N'
    else:
        return 'N'  # Devuelve un valor nulo si el sector no es 'BEAUTY'

completed_combined_df['hashtag_beauty'] = completed_combined_df.apply(verify_hastags, axis=1)

In [138]:
completed_combined_df['hashtag_beauty'].value_counts(dropna=False)

N    9761
Y    3523
Name: hashtag_beauty, dtype: int64

In [139]:
completed_combined_df['hashtag_beauty'].value_counts(normalize=True)

N    0.734794
Y    0.265206
Name: hashtag_beauty, dtype: float64

#### Hashtags Fitness

In [ ]:
# Definimos la lista de palabras clave del sector fitness

keywords = ["fit", "workout", "entrena", "gym", "gim", "exercise", "ejercicio", "health", "salud", "wellness", "bienestar", "training",
                  "strength", "fuerza","cardio", "nutricion", "nutrition", "diet", "body", "cuerp", "weight", "peso", "yoga", "pilates", "run",
                  "cross", "corr","sudor", "metas", "músculo", "active", "sweat", "goals", "progres", "muscle", "endurance", "flexiones", "ectomorfo"
                  "aerobic", "bootcamp", "calist", "conditioning", "core", "deadlift", "levantar", "repeticiones", "hydrat", "routin", "mesomorfo",
                  "flexib", "hiit", "interval", "kettlebell", "lift", "mobili", "plank", "plyometrics", "sinazúcar", "suplement", "pullups",
                  "reps", "resist", "spin", "sports", "deportes", "sentadillas", "squats", "stability", "stamina", "stretch", "tone", "endomorfo",
                  "trx", "zumba", "abs", "glute", "vegan", "keto", "paleo", "lowcarb", "glutenfree", "estabilidad", "pesas", "rutin", "grasa",
                  "organic", "protein", "detox", "cleaneating", "eatclean", "mealprep", "water", "estira", "bajocarb", "singluten", "muscul",
                  "fiber", "vitamin", "antioxidants", "omega3", "probiotics", "food", "comida", "come", "sana", "preparación", "receta", "tonific",
                  "plantbased", "dairyfree", "sugarfree", "raw", "vegetarian", "hidrat", "pushups", "espalda", "pierna", "bícep", "trícep", "pectora"]

def verify_hastags(row):
    if row['sector'] == 'FITNESS':
        for keyword in keywords:
            if any(re.search(rf"{keyword}", hashtag, re.IGNORECASE) for hashtag in row['hashtags']):
                return 'Y'
        else:
            return 'N'
    else:
        return 'N'  # Devuelve un valor nulo si el sector no es 'FITNESS'
    
completed_combined_df['hashtag_fitness'] = completed_combined_df.apply(verify_hastags, axis=1)


In [141]:
completed_combined_df['hashtag_fitness'].value_counts(normalize=True)

N    0.662301
Y    0.337699
Name: hashtag_fitness, dtype: float64

#### Perimetro

In [152]:
#Creamos ahora la variable 'perimetro' para identificar todos los registros que se consideran que son publicaciones de contenido 
#perteneciente a los sectores objeto de estudio.

completed_combined_df['perimetro'] = 'N'
completed_combined_df.loc[(completed_combined_df['sector'] == 'BEAUTY') & (completed_combined_df['hashtag_beauty'] == 'Y'), 'perimetro'] = 'Y'
completed_combined_df.loc[(completed_combined_df['sector'] == 'FITNESS') & (completed_combined_df['hashtag_fitness'] == 'Y'), 'perimetro'] = 'Y'


In [155]:
completed_combined_df['perimetro'].value_counts(normalize=True)

Y    0.602906
N    0.397094
Name: perimetro, dtype: float64

#### Hashtags Publicidad

In [156]:
# Definimos la lista de palabras clave
keywords = ['ad', 'publi', 'advertisement', 'colaboracion', 'colab', 'publicidad']

# Definimos una función para verificar si alguna palabra clave está presente en el texto
def verify_hastags(hashtags):
    for keyword in keywords:
        if any(re.search(rf"\b{keyword}\b", hashtag, re.IGNORECASE) for hashtag in hashtags):
            return 'Y'
    return 'N'

# Aplicamos la función a la columna 'hashtags' y creamos el campo 'hashtag_publi'
completed_combined_df['hashtag_publi'] = completed_combined_df['caption_text'].apply(lambda x: verify_hastags([hashtag.lower() for hashtag in x.split()]))


In [157]:
completed_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13284 entries, 0 to 13283
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   _id                      13284 non-null  object  
 1   username                 13284 non-null  object  
 2   typename                 13284 non-null  category
 3   followers                13284 non-null  int64   
 4   following                13284 non-null  int64   
 5   verified                 13284 non-null  bool    
 6   count_comments           13284 non-null  int64   
 7   count_likes              13284 non-null  int64   
 8   caption_text             13284 non-null  object  
 9   has_audio                13284 non-null  bool    
 10  comments_disabled        13284 non-null  bool    
 11  post_type                6071 non-null   category
 12  timestamp                13284 non-null  object  
 13  is_professional_account  13284 non-null  bool    
 14  catego

#### Engagement rate

El engagement rate, o tasa de participación, es una métrica de análisis de redes sociales para medir el nivel de interacción y participación de los seguidores con el contenido publicado por una cuenta o perfil.

Se puede calcular como un porcentaje que representa la cantidad de interacciones (como me gusta, comentarios) dividida por el número total de seguidores y multiplicado por 100.

In [158]:
completed_combined_df['engagement_rate_1'] = ((completed_combined_df['count_likes'] + completed_combined_df['count_comments']) / completed_combined_df['followers']) * 100
completed_combined_df['engagement_rate_2'] = ((completed_combined_df['count_likes']) / completed_combined_df['followers']) * 100

In [159]:
completed_combined_df.head()

,_id,username,typename,followers,following,verified,count_comments,count_likes,caption_text,has_audio,...,Date,Hour,hashtags,hashtag_beauty,hashtag_fitness,hashtag_publi,engagement_rate_1,engagement_rate_2,len_text,perimetro
0,1461031291230411282,bazardemarisse,GraphImage,1879,1696,False,1,43,"Son pendrives aunque no lo parezcan y con ellos empiezo la #guerradependrives con @anitacamaram. Te apuntas? Sólo tienes que subir los pendrives más originales, estrafalarios, atrevidos, subversivos o que más te gustes que tengas. Y la ganadora se lleva un premio! Empieza la guerraaaaa 💪🏼💪🏼 👓🚬👄🍬#pendrives #beautyblog #fashionblog #blogdebelleza #blogdemoda #lifestyleblog #blogger #momultiopticas #oralb #bellemakeup #justcavalli #it",False,...,2017-03-01,17:15:23,"[guerradependrives, pendrives, beautyblog, fashionblog, blogdebelleza, blogdemoda, lifestyleblog, blogger, momultiopticas, oralb, bellemakeup, justcavalli, it]",Y,N,N,2.341671,2.288451,435,Y
1,1461057228236014541,bazardemarisse,GraphImage,1879,1696,False,2,49,"Hoy hemos conocido un estudio muy interesante de @nivea_es sobre la percepción personal y del entorno de las mujeres españolas en torno a la cincuentena. Y todo junto a tres embajadoras de lujo: María Escario, @josetoledoficial y @cayetanagc 💗 #blogdebelleza #beautyblog #meatrevoaseryo #nivea #womencare #skincare #pressday",False,...,2017-03-01,18:06:55,"[blogdebelleza, beautyblog, meatrevoaseryo, nivea, womencare, skincare, pressday]",Y,N,N,2.714210,2.607770,324,Y
2,1461626070960120583,bazardemarisse,GraphImage,1879,1696,False,1,61,Celebrando los 25 años de @uriagespain en sitios bonitos. Felicidades! Y a por otros 25 años más! 👯 🎉🎂 #beautyblog #blogdebelleza #uriage #felizcumpleaños #beauty #beautycare #flowershop #sitiosconencanto #madridlifestyle #madridshopping #happybirthday,False,...,2017-03-02,12:57:07,"[beautyblog, blogdebelleza, uriage, felizcumpleaños, beauty, beautycare, flowershop, sitiosconencanto, madridlifestyle, madridshopping, happybirthday]",Y,N,N,3.299627,3.246408,252,Y
3,1462547320532230958,bazardemarisse,GraphImage,1879,1696,False,6,68,Una gama muy amplia y un colorido impresionante. Así es la nueva firma de maquillaje que llega a España: @bronx_colors. Por ahora sólo está disponible en @pprimor seleccionadas. Cuando pruebes sus productos os lo contaré con todo detalle 🤙🏻💄💅🏼 #beautyblog #blogdebelleza #bronxcolors #makeup #new #igersbeauty #maquillaje #nails #naillacquer #lipstick,False,...,2017-03-03,19:27:28,"[beautyblog, blogdebelleza, bronxcolors, makeup, new, igersbeauty, maquillaje, nails, naillacquer, lipstick]",Y,N,N,3.938265,3.618946,351,Y
4,1463898105857794613,bazardemarisse,GraphImage,1879,1696,False,10,76,Las mejores pizzas de Madrid están al lado del Bernabeu y son de @kilometrosdepizza. Hoy hemos probado la Rolling y La venta. Espectaculares! 😋😋🍕🍕 #lifestyleblog #lifestyle #restaurantemadrid #pizzamadrid #pizza #kilometrosdepizza #italianfood #igersfood #foodiegram #foodies #igersfoodie,False,...,2017-03-05,16:11:14,"[lifestyleblog, lifestyle, restaurantemadrid, pizzamadrid, pizza, kilometrosdepizza, italianfood, igersfood, foodiegram, foodies, igersfoodie]",Y,N,N,4.576903,4.044705,288,Y


#### Longitud del texto

In [160]:
completed_combined_df['len_text'] = completed_combined_df['caption_text'].apply(lambda x: len(str(x)))

In [161]:
completed_combined_df

,_id,username,typename,followers,following,verified,count_comments,count_likes,caption_text,has_audio,...,Date,Hour,hashtags,hashtag_beauty,hashtag_fitness,hashtag_publi,engagement_rate_1,engagement_rate_2,len_text,perimetro
0,1461031291230411282,bazardemarisse,GraphImage,1879,1696,False,1,43,"Son pendrives aunque no lo parezcan y con ellos empiezo la #guerradependrives con @anitacamaram. Te apuntas? Sólo tienes que subir los pendrives más originales, estrafalarios, atrevidos, subversivos o que más te gustes que tengas. Y la ganadora se lleva un premio! Empieza la guerraaaaa 💪🏼💪🏼 👓🚬👄🍬#pendrives #beautyblog #fashionblog #blogdebelleza #blogdemoda #lifestyleblog #blogger #momultiopticas #oralb #bellemakeup #justcavalli #it",False,...,2017-03-01,17:15:23,"[guerradependrives, pendrives, beautyblog, fashionblog, blogdebelleza, blogdemoda, lifestyleblog, blogger, momultiopticas, oralb, bellemakeup, justcavalli, it]",Y,N,N,2.341671,2.288451,435,Y
1,1461057228236014541,bazardemarisse,GraphImage,1879,1696,False,2,49,"Hoy hemos conocido un estudio muy interesante de @nivea_es sobre la percepción personal y del entorno de las mujeres españolas en torno a la cincuentena. Y todo junto a tres embajadoras de lujo: María Escario, @josetoledoficial y @cayetanagc 💗 #blogdebelleza #beautyblog #meatrevoaseryo #nivea #womencare #skincare #pressday",False,...,2017-03-01,18:06:55,"[blogdebelleza, beautyblog, meatrevoaseryo, nivea, womencare, skincare, pressday]",Y,N,N,2.714210,2.607770,324,Y
2,1461626070960120583,bazardemarisse,GraphImage,1879,1696,False,1,61,Celebrando los 25 años de @uriagespain en sitios bonitos. Felicidades! Y a por otros 25 años más! 👯 🎉🎂 #beautyblog #blogdebelleza #uriage #felizcumpleaños #beauty #beautycare #flowershop #sitiosconencanto #madridlifestyle #madridshopping #happybirthday,False,...,2017-03-02,12:57:07,"[beautyblog, blogdebelleza, uriage, felizcumpleaños, beauty, beautycare, flowershop, sitiosconencanto, madridlifestyle, madridshopping, happybirthday]",Y,N,N,3.299627,3.246408,252,Y
3,1462547320532230958,bazardemarisse,GraphImage,1879,1696,False,6,68,Una gama muy amplia y un colorido impresionante. Así es la nueva firma de maquillaje que llega a España: @bronx_colors. Por ahora sólo está disponible en @pprimor seleccionadas. Cuando pruebes sus productos os lo contaré con todo detalle 🤙🏻💄💅🏼 #beautyblog #blogdebelleza #bronxcolors #makeup #new #igersbeauty #maquillaje #nails #naillacquer #lipstick,False,...,2017-03-03,19:27:28,"[beautyblog, blogdebelleza, bronxcolors, makeup, new, igersbeauty, maquillaje, nails, naillacquer, lipstick]",Y,N,N,3.938265,3.618946,351,Y
4,1463898105857794613,bazardemarisse,GraphImage,1879,1696,False,10,76,Las mejores pizzas de Madrid están al lado del Bernabeu y son de @kilometrosdepizza. Hoy hemos probado la Rolling y La venta. Espectaculares! 😋😋🍕🍕 #lifestyleblog #lifestyle #restaurantemadrid #pizzamadrid #pizza #kilometrosdepizza #italianfood #igersfood #foodiegram #foodies #igersfoodie,False,...,2017-03-05,16:11:14,"[lifestyleblog, lifestyle, restaurantemadrid, pizzamadrid, pizza, kilometrosdepizza, italianfood, igersfood, foodiegram, foodies, igersfoodie]",Y,N,N,4.576903,4.044705,288,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13279,3111938987433797684,vikikacosta,GraphVideo,1062361,983,True,59,14286,🦀✅VACACIONES???🙌🏼🎉 Haz este entrenamiento SIN MATERIAL 🙌🏼❤️ recuerda guardarlo 😄 👌🏼Recomendación: hacer 2-3 rondas de los ejercicios 🚀,True,...,2023-05-27,12:57:31,[],N,N,N,1.350294,1.344741,134,N
13280,3112105038420076511,vikikacosta,GraphVideo,1062361,983,True,131,10653,Irene & Manu 27/05/23 📍Mallorca El vestido de mis sueños @oficial_pero,True,...,2023-05-27,18:29:01,[],N,N,N,1.015098,1.002766,70,N
13281,3112645464515939340,vikikacosta,GraphSidecar,1062361,983,True,58,11523,"Ayer fue mágico, no lo digo por decir. Irene y Manu son personas vitamina, que se esfuerzan por perseguir sus sueños, por difíciles que resulten. Son de esas personas qu

## Almacenamiento en formato CSV

En la sección anterior ya se dispone de todo lo necesario para obtener un dataframe final que posea todos los datos que se desean extraer del Datalake, así que a continuación se presenta como salvar estos datos a un formato CSV.

La elección de este tipo de fichero se debe **a que en el futuro serán visualizados los datos en PowerBI** y para evitar cualquier problema en la carga de la información se ha optado por una extesión con la que se acostumbre a trabajar en esta herramienta

In [162]:
# Especificamos la ruta de destino en GCS
BUCKET_NAME = 'unir_bucket_tfm'
PROJECT_ID = 'tfminfluencers'
DESTINATION_FOLDER = 'TABLA_FINAL'
DESTINATION_FILE = 'completed_combined_df.csv'
DESTINATION_PATH = f'{BUCKET_NAME}/{DESTINATION_FOLDER}/{DESTINATION_FILE}'

In [163]:
# Configuramos las opciones de visualización para mostrar el contenido completo
pd.set_option('display.max_colwidth', None)

# Reemplazamos los saltos de línea y caracteres especiales en el texto
completed_combined_df['caption_text'] = completed_combined_df['caption_text'].str.replace('\n', ' ')
completed_combined_df['caption_text'] = completed_combined_df['caption_text'].str.replace('\r', ' ')
completed_combined_df['caption_text'] = completed_combined_df['caption_text'].str.replace(';', ',')

# Reemplazamos los saltos de línea en el campo "biography"
completed_combined_df['biography'] = completed_combined_df['biography'].str.replace('\n', ' ')


# Guardamos el DataFrame en un archivo CSV localmente
completed_combined_df.to_csv(DESTINATION_FILE, index=False, quoting=csv.QUOTE_ALL, encoding='utf-8-sig', escapechar='\\')

# Crear una instancia del sistema de archivos de GCS
fs = gcsfs.GCSFileSystem(project=PROJECT_ID)

# Mover el archivo CSV al bucket y carpeta deseada
with fs.open(DESTINATION_PATH, 'wb') as f:
    with open(DESTINATION_FILE, 'rb') as local_file:
        f.write(local_file.read())

# Eliminar el archivo local después de moverlo a GCS
os.remove(DESTINATION_FILE)

print("Archivo CSV guardado correctamente en GCS.")

Archivo CSV guardado correctamente en GCS.


## Crear tabla en Big Query a partir de CSV

In [150]:
# !pip install bigquery

In [164]:
from google.cloud import bigquery

# Especificamos los valores
DATASET_ID = 'influencer_post_table'
FILE_NAME = f'{DESTINATION_FOLDER}/{DESTINATION_FILE}'
TABLE_ID = 'completed_combined_df'

# Creamos el dataset en BigQuery
client = bigquery.Client(project=PROJECT_ID)
dataset_ref = client.dataset(DATASET_ID)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'  # Especifica la ubicación del dataset

dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Se ha creado el dataset '{DATASET_ID}' en BigQuery.")

# Cargamos el archivo CSV en una nueva tabla
table_ref = dataset_ref.table(TABLE_ID)
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

uri = f"gs://{BUCKET_NAME}/{FILE_NAME}"
load_job = client.load_table_from_uri(uri, table_ref, job_config=job_config)

load_job.result()  # Esperamos a que se complete la carga

print(f"Se ha cargado el archivo CSV en la tabla '{TABLE_ID}' en BigQuery.")

Se ha creado el dataset 'influencer_post_table' en BigQuery.
Se ha cargado el archivo CSV en la tabla 'completed_combined_df' en BigQuery.


In [133]:
completed_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13284 entries, 0 to 13283
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   _id                      13284 non-null  object  
 1   username                 13284 non-null  object  
 2   typename                 13284 non-null  category
 3   followers                13284 non-null  int64   
 4   following                13284 non-null  int64   
 5   verified                 13284 non-null  bool    
 6   count_comments           13284 non-null  int64   
 7   count_likes              13284 non-null  int64   
 8   caption_text             13284 non-null  object  
 9   has_audio                13284 non-null  bool    
 10  comments_disabled        13284 non-null  bool    
 11  post_type                6071 non-null   category
 12  timestamp                13284 non-null  object  
 13  is_professional_account  13284 non-null  bool    
 14  catego